In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.font_manager

In [2]:
fname = os.listdir('results')
problem = 'lava_problem'
npz_name = []
for i in fname:
    # if i[-5] == 'f':
    if 'Kullback' in i:
        npz_name.append(i)
    elif 'Total' in i:
        npz_name.append(i)
    elif 'Neyman' in i:
        npz_name.append(i)

# Load x-axis and POMDP data
opt_data = np.load("results/"+problem + "_optimal_results.npz")
p_correct_vals = opt_data['p_correct_vals']
opt_values = opt_data['opt_values']

data_all = [opt_values]

for j in npz_name:
    data = np.load("results/"+j)
    bounds = data['bounds']
    data_all.append(data['bounds'])

In [3]:
# Plot
# fig, ax = plt.subplots()
# ax.plot(p_correct_vals, opt_values, '*--', label='POMDP', linewidth=0.5)

plt.rcParams["figure.figsize"] = (15,3)
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times']
plt.rcParams['text.usetex'] = True


rows = ('POMDP', 'Neyman', 'TV', 'KL')
colors = ['#005a9c','white','white','white']

n_rows = len(data_all)
bar_width = 0.03

for row in [3, 0]:
    plt.bar(p_correct_vals, data_all[row], bar_width, edgecolor = 'k', color=colors[row])

plt.xlabel('$p_{\text{correct}}$')
plt.ylabel('Reward')
plt.yticks([1,3,5])
plt.show()

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 1080x216 with 1 Axes>

In [ ]:

columns = ('Freeze', 'Wind', 'Flood', 'Quake', 'Hail')
rows = ['%d year' % x for x in (100, 50, 20, 10, 5)]

values = np.arange(0, 2500, 500)
value_increment = 1000

# Get some pastel shades for the colors
colors = plt.cm.BuPu(np.linspace(0, 0.5, len(rows)))
n_rows = len(data_all)

index = np.arange(len(columns)) + 0.3
bar_width = 0.4

# Initialize the vertical-offset for the stacked bar chart.
y_offset = np.zeros(len(columns))

# Plot bars and create text labels for the table
cell_text = []
for row in range(n_rows):
    plt.bar(index, data[row], bar_width, bottom=y_offset, color=colors[row])
    y_offset = y_offset + data[row]
    cell_text.append(['%1.1f' % (x / 1000.0) for x in y_offset])
# Reverse colors and text labels to display the last value at the top.
colors = colors[::-1]
cell_text.reverse()

# Add a table at the bottom of the axes
the_table = plt.table(cellText=cell_text,
                      rowLabels=rows,
                      rowColours=colors,
                      colLabels=columns,
                      loc='bottom')

# Adjust layout to make room for the table:
plt.subplots_adjust(left=0.2, bottom=0.2)

plt.ylabel("Loss in ${0}'s".format(value_increment))
plt.yticks(values * value_increment, ['%d' % val for val in values])
plt.xticks([])
plt.title('Loss by Disaster')

plt.show()